There are a few simple things we cna do to help our API become a little simpler. This is in absence of using multiple-dispatch. 

So, to show the new polymorphic weights constructors in action, I'd like to show a decorator I'd like to place in `common`, the `intercept_filepath` decorator.

In [1]:
import pysal as ps

In [2]:
datapath = ps.examples.get_path('south.shp')

In [3]:
df = ps.pdio.read_files(datapath)
file_handler = ps.open(datapath)

This could be used library-wide to enable what might look like multiple dispatch at first, but is, in fact, a simple way to preparse arguments to match the API of the defined function. 

In short, the `intercept_filepath` decorator intercepts the arguments that are passed to its function. First, it checks the type of the function's first argument. If it's a string, it attempts to open it using `pysal.open` before sending it to the original function. 

In [4]:
from pysal.common import intercept_filepath

To show, this is a simple function that expcets a geometry collection:

In [5]:
def a(coll, k):
    """
    Print the first k centroids from a geometry collection
    """
    print([coll[i].centroid for i in range(k)])

Undecorated, it will not work if a string is passed:

In [6]:
#a(datapath, 2) #I will fail if you uncomment me

But, will work if a file handler or dataframe column is passed:

In [7]:
a(df.geometry, 1)
a(file_handler, 4)

[(-80.578701723812, 40.520355179265735)]
[(-80.578701723812, 40.520355179265735), (-80.57987548953308, 40.27464887574844), (-80.62260358646088, 40.09987235391358), (-80.66606152758811, 39.86106340922978)]


But, if we decorate the function, we can preprocess the arugments:

In [8]:
a = intercept_filepath(a) # decorators are just functions that return functions

In [9]:
print(datapath)
a(datapath, 1)

pysal/examples/south/south.shp
[(-80.578701723812, 40.520355179265735)]


Using this, we can define a target API that *only* constructs weights from iterables. Then, if we decorate that function, we can preparse its arguments to get them in the form the API expects.

This is a much cleaner way to do the dispatch-table argument parsing, since we can just centralize **all** deployments of the pattern:
```python
elif isinstance(x, str):
    x = open(x)
```
in the dispatch table. 

So, in future, this could mean that any API functions that we want to be very polymorphic look something like:

```python
@intercept_filepath
@intercept_dataframe
def Rook(polygon_iterable, transformation='r', **kw):
    return do_weights()
```
This could also work for enforcing consistent array shaping/casting rules, but I will be investigating this going foward. 

### Right now

This means that the polymorphic weights stuff is already done:

In [10]:
from pysal.weights import user2 as newW

In [12]:
rW0 = ps.rook_from_shapefile(datapath).full()[0]
rW1 = newW.Rook(df).full()[0]
rW2 = newW.Rook(file_handler).full()[0]
rW3 = newW.Rook(datapath).full()[0]

In [13]:
from numpy.testing import assert_allclose

In [14]:
assert_allclose(rW0, rW1)
assert_allclose(rW0, rW2)
assert_allclose(rW0, rW3)